In [1]:
# To find values within a column...
#df.loc[df['column_name'] == some_value]
#x = dfs[0][1]
#x.loc[x["ncli"] == 8151]

In [2]:
import os
import re
import math
import pandas as pd
import numpy as np

import plotly
import plotly.plotly as py
from plotly.graph_objs import *

plotly.tools.set_credentials_file(username='theano', api_key='hkYRYxjyM0aAGA5nmGq1')

In [3]:
import matplotlib.pyplot as plt

import matplotlib
matplotlib.style.use('ggplot')

In [4]:
home = os.getcwd()
print(home)
sorted([x for x in os.listdir(home)])
collectl_stats = ['Date', '[CPU]User%', '[CPU]Nice%', '[CPU]Sys%', '[CPU]Wait%',
                               '[CPU]Irq%', '[CPU]Soft%', '[CPU]Steal%', '[CPU]Idle%',
                               '[CPU]Totl%', '[CPU]Intrpt/sec', '[CPU]Ctx/sec', '[CPU]Proc/sec',
                               '[CPU]ProcQue', '[CPU]ProcRun', '[CPU]L-Avg1', '[CPU]L-Avg5',
                               '[CPU]L-Avg15', '[CPU]RunTot', '[CPU]BlkTot']
loaded_files = []


/Users/theano/mountpoint/tsch/mininetpipe/results/2017-12-18-18-44


In [5]:
def load_datfile(fpath):
    # Returns a dataframe
    df = pd.read_csv(fpath, 
                     index_col=False,
                     names=["ncli","throughput"])
    return df


def load_collectl(fname):
    df = pd.read_csv(fname,
                     delim_whitespace=True,
                     comment="#",
                     names=collectl_stats)
    return df

def load_data_and_stats(basename):
    pds = []
    prog = re.compile("%s_\w+\.dat$" % basename)
    
    for fname in [x for x in os.listdir(".") if prog.match(x)]:
        df_tput = load_datfile(os.path.join(home, fname))
        
        print("Loading file %s" % fname)
        collectl_fname = [x for x in os.listdir(".") if "%s-server-0" % fname in x]
        if len(collectl_fname) > 0:
            df_stats = load_collectl(collectl_fname[0])
            newdf = pd.concat([df_tput.sum(), df_stats.mean()])
        else:
            newdf = df_tput.sum()
        
        nclients = re.search("(?<=_c)\d+", fname)
        try:
            nclients = int(nclients.group(0))
            if newdf["ncli"] != nclients:
                print("skipping df with %d clients...\n" % newdf["ncli"])
                continue
        except:
                print("exception")
                continue
            
        expduration = int(re.search("(?<=_u)\d+", fname).group(0))
        
        newdf["fname"] = fname
        newdf["expduration"] = expduration
        newdf["numa"] = int(fname.split("numa")[0])        
        pds.append(newdf)
        
    return (basename, (pd.concat(pds, axis=1)
                        .transpose()).apply(pd.to_numeric, errors='coerce'))

def load_data_no_sum(basename):
    pds = []
    prog = re.compile("%s_\w+\.dat$" % basename)
    
    for fname in [x for x in os.listdir(".") if prog.match(x)]:
        df_tput = load_datfile(os.path.join(home, fname))
        
        expduration = int(re.search("(?<=_u)\d+", fname).group(0))
        
        collectl_fname = [x for x in os.listdir(".") if "%s-server-0" % fname in x]
        
        if len(collectl_fname) > 0:
            df_stats = load_collectl(collectl_fname[0])
            newdf = pd.concat([df_tput, df_stats])
        else:
            newdf = df_tput
        
        newdf["expduration"] = expduration
        newdf["fname"] = fname  # for finding weird data
        newdf["ncores"] = int(fname.split("cores")[0])
        pds.append(newdf)

    return (basename, pd.concat(pds).apply(pd.to_numeric, errors='coerce'))

In [6]:
dfs = {}
os.listdir(".")

['1numa_u80_c2048_16.dat-server-0-20171219.tab',
 '2numa_u80_c2048_23.dat-server-0-20171218.tab',
 '2numa_u80_c2048_21.dat',
 '2numa_u80_c2048_26.dat',
 '3numa_u80_c2048_20.dat-server-0-20171218.tab',
 '1numa_u80_c2048_3.dat-server-0-20171219.tab',
 '3numa_u80_c2048_8.dat-server-0-20171218.tab',
 '3numa_u80_c2048_12.dat',
 '2numa_u80_c2048_28.dat',
 '4numa_u80_c2048_16.dat-server-0-20171218.tab',
 '2numa_u80_c2048_4.dat-server-0-20171218.tab',
 '3numa_u80_c2048_15.dat',
 '3numa_u80_c2048_20.dat',
 '3numa_u80_c2048_23.dat-server-0-20171218.tab',
 '4numa_u80_c2048_1.dat',
 '1numa_u80_c2048_0.dat-server-0-20171219.tab',
 '3numa_u80_c2048_8.dat',
 '3numa_u80_c2048_27.dat',
 '1numa_u80_c2048_29.dat-server-0-20171219.tab',
 '4numa_u80_c2048_6.dat',
 '1numa_u80_c2048_15.dat-server-0-20171219.tab',
 '2numa_u80_c2048_20.dat-server-0-20171218.tab',
 '2numa_u80_c2048_7.dat-server-0-20171218.tab',
 '3numa_u80_c2048_29.dat',
 '2numa_u80_c2048_3.dat',
 '2numa_u80_c2048_13.dat',
 '4numa_u80_c2048_8.d

In [11]:
for exp in [
    "1numa",
    "2numa",
    "3numa",
    "4numa"
]:
    print ("Working on experiment %s..." % exp)
    label, df = load_data_and_stats(exp)
    df = df.fillna(df.mean())
    
    # Remove rows where ncli != 2048
    dfs[exp] = df
    

Working on experiment 1numa...
Loading file 1numa_u80_c2048_5.dat
Loading file 1numa_u80_c2048_2.dat
Loading file 1numa_u80_c2048_19.dat
Loading file 1numa_u80_c2048_10.dat
Loading file 1numa_u80_c2048_17.dat
Loading file 1numa_u80_c2048_22.dat
Loading file 1numa_u80_c2048_25.dat
Loading file 1numa_u80_c2048_3.dat
Loading file 1numa_u80_c2048_4.dat
Loading file 1numa_u80_c2048_24.dat
Loading file 1numa_u80_c2048_23.dat
Loading file 1numa_u80_c2048_16.dat
Loading file 1numa_u80_c2048_11.dat
Loading file 1numa_u80_c2048_18.dat
Loading file 1numa_u80_c2048_29.dat
Loading file 1numa_u80_c2048_20.dat
Loading file 1numa_u80_c2048_27.dat
Loading file 1numa_u80_c2048_12.dat
Loading file 1numa_u80_c2048_15.dat
Loading file 1numa_u80_c2048_7.dat
Loading file 1numa_u80_c2048_0.dat
Loading file 1numa_u80_c2048_9.dat
Loading file 1numa_u80_c2048_14.dat
Loading file 1numa_u80_c2048_13.dat
Loading file 1numa_u80_c2048_26.dat
Loading file 1numa_u80_c2048_21.dat
Loading file 1numa_u80_c2048_28.dat
Load

In [12]:
grouped = pd.concat([df for key, df in dfs.items()]).groupby("numa")
agged = grouped.agg([np.std, np.mean])

In [13]:
grouped.get_group(1)

,ncli,throughput,[CPU]User%,[CPU]Nice%,[CPU]Sys%,[CPU]Wait%,[CPU]Irq%,[CPU]Soft%,[CPU]Steal%,[CPU]Idle%,...,[CPU]ProcQue,[CPU]ProcRun,[CPU]L-Avg1,[CPU]L-Avg5,[CPU]L-Avg15,[CPU]RunTot,[CPU]BlkTot,fname,expduration,numa
0,2048.0,4.353963e+05,0.687500,0.0,11.537500,0.00,0.0,5.762500,0.0,82.125000,...,692.000000,4.200000,2.943250,3.001750,2.249750,3.562500,0.000000,NaN,80,1
1,2048.0,inf,0.800000,0.0,14.750000,0.00,0.0,7.200000,0.0,77.212500,...,692.000000,4.850000,3.142625,2.683375,1.832500,4.437500,0.000000,NaN,80,1
2,2048.0,inf,0.687500,0.0,11.987500,0.00,0.0,5.850000,0.0,81.512500,...,692.000000,4.062500,3.895500,3.235375,2.958750,3.325000,0.000000,NaN,80,1
3,2048.0,inf,0.800000,0.0,14.112500,0.00,0.0,7.175000,0.0,77.875000,...,693.000000,4.475000,3.206875,3.451875,2.828250,4.137500,0.000000,NaN,80,1
4,2048.0,5.547318e+05,0.870130,0.0,15.597403,0.00,0.0,7.688312,0.0,75.740260,...,692.103896,5.012987,3.262597,2.966234,2.839091,4.376623,0.012987,NaN,80,1
5,2048.0,inf,0.762500,0.0,14.450000,0.00,0.0,6.862500,0.0,77.837500,...,692.000000,4.850000,3.388625,3.288000,3.052250,4.325000,0.000000,NaN,80,1
6,2048.0,inf,0.812500,0.0,15.362500,0.00,0.0,7.112500,0.0,76.600000,...,692.000000,5.187500,3.278250,3.017875,2.977000,4.537500,0.000000,NaN,80,1
7,2048.0,inf,0.762500,0.0,12.637500,0.00,0.0,6.137500,0.0,80.500000,...,692.000000,4.600000,3.668125,3.019625,2.046875,3.900000,0.000000,NaN,80,1
8,2048.0,5.791539e+05,0.725000,0.0,14.200000,0.00,0.0,6.987500,0.0,78.025000,...,691.937500,4.512500,3.153750,3.019375,2.161250,3.987500,0.000000,NaN,80,1
9,2048.0,inf,0.775000,0.0,14.175000,0.00,0.0,6.850000,0.0,78.162500,...,692.000000,4.487500,2.806375,2.936250,2.952625,3.912500,0.000000,NaN,80,1


In [14]:
data = []

barwidth = 0.5

x = [numacount + " node(s)" for numacount in 
    agged.index.astype(int).astype(str)]

# Throughput
data.append(Bar(
    x = x,
    y = agged[("[CPU]Sys%", "mean")],
    yaxis = 'y2',
    name = 'CPU Sys%',
    width = barwidth,
    opacity=0.6
))
data.append(Bar(
    x = x,
    y = agged[("[CPU]Soft%", "mean")],
    yaxis = 'y2',
    name = 'CPU Soft%',
    width = barwidth,
    opacity=0.6
))
data.append(Bar(
    x = x,
    y = agged[("[CPU]User%", "mean")],
    yaxis = 'y2',
    name = 'CPU User%',
    width = barwidth,
    opacity=0.6
))

for label, df in grouped:
    data.append(Box(
        y = df["throughput"],
        name = "%s node(s)" % label,
        boxpoints = 'all',
        #boxmean = "sd",
        showlegend = False,
        marker = dict(color = "rgb(8, 81, 156)",
                     size = 3),
        jitter = 0.5,
    ))

    
layout = Layout (
    xaxis = {'title': 'number of NUMA nodes'},
    yaxis = {'title': 'throughput (pps)', 'rangemode': 'tozero'},
    yaxis2 = {'title': 'CPU utilization', 
              'side': 'right', 
              'overlaying': 'y',
              'showgrid': False},
    title = 'Small-packet throughput with varying NUMA count: 2048 clients',
    #legend = {'x': 0.01, 'y': 1},
    #boxmode = "group"
    barmode = "stack",
    boxgap = 0.5
)

fig = Figure(data=data, layout=layout)
py.iplot(fig, filename='tcp-cpu-throughput-varynuma-2048cli')

In [32]:
ratios = []
ratios.append(100*agged[("[CPU]User%", "mean")]/agged[("[CPU]Totl%", "mean")])
ratios.append(100*agged[("[CPU]Soft%", "mean")]/agged[("[CPU]Totl%", "mean")])
ratios.append(100*agged[("[CPU]Sys%", "mean")]/agged[("[CPU]Totl%", "mean")])
ratios

[numa
 4    4.455216
 dtype: float64, numa
 4    27.948884
 dtype: float64, numa
 4    66.858389
 dtype: float64]

In [24]:
data = []

i = 1
for label, df in grouped:
    data.append(Histogram(x=df["throughput"],
                          nbinsx=10,
                          name="%s node(s)" % label,
                          opacity=0.6,
                          histnorm="probability"))

layout = Layout (
    xaxis = {'title': 'throughput (pps)'},
    yaxis = {'title': 'frequency'},
    title = 'Throughput frequency for varying core counts',
    showlegend = True,
) 
fig = Figure(data=data, layout=layout)
    
py.iplot(fig, filename="tput_frequency_durations_ncores")